# Vitessce Widget Tutorial

# Visualization of a SpatialData object

## Import dependencies


In [16]:
import os
from os.path import join, isfile, isdir
from urllib.request import urlretrieve
import zipfile
import shutil

from vitessce import (
    VitessceConfig,
    ViewType as vt,
    CoordinationType as ct,
    CoordinationLevel as CL,
    SpatialDataWrapper,
    get_initial_coordination_scope_prefix
)

In [17]:
data_dir = "data"
zip_filepath = join(data_dir, "xenium_rep1_io.spatialdata.zarr.zip")
spatialdata_filepath = join(data_dir, "xenium_rep1_io.spatialdata.zarr")

In [21]:
if not isdir(spatialdata_filepath):
    if not isfile(zip_filepath):
        os.makedirs(data_dir, exist_ok=True)
        urlretrieve('https://s3.embl.de/spatialdata/spatialdata-sandbox/xenium_rep1_io.zip', zip_filepath)
    with zipfile.ZipFile(zip_filepath,"r") as zip_ref:
        zip_ref.extractall(data_dir)
        os.rename(join(data_dir, "data.zarr"), spatialdata_filepath)
        
        # This Xenium dataset has an AnnData "raw" element.
        # Reference: https://github.com/giovp/spatialdata-sandbox/issues/55
        raw_dir = join(spatialdata_filepath, "tables", "table", "raw")
        if isdir(raw_dir):
            shutil.rmtree(raw_dir)

In [22]:
from spatialdata import read_zarr

In [23]:
sdata = read_zarr(spatialdata_filepath)
sdata

version mismatch: detected: RasterFormatV02, requested: FormatV04
/Users/mkeller/research/dbmi/vitessce/vitessce-python/.venv/lib/python3.10/site-packages/zarr/creation.py:614: UserWarning: ignoring keyword argument 'read_only'
  compressor, fill_value = _kwargs_compat(compressor, fill_value, kwargs)
version mismatch: detected: RasterFormatV02, requested: FormatV04


SpatialData object, with associated Zarr store: /Users/mkeller/research/dbmi/vitessce/vitessce-python/docs/notebooks/data/xenium_rep1_io.spatialdata.zarr
├── Images
│     ├── 'morphology_focus': DataTree[cyx] (1, 25778, 35416), (1, 12889, 17708), (1, 6444, 8854), (1, 3222, 4427), (1, 1611, 2213)
│     └── 'morphology_mip': DataTree[cyx] (1, 25778, 35416), (1, 12889, 17708), (1, 6444, 8854), (1, 3222, 4427), (1, 1611, 2213)
├── Points
│     └── 'transcripts': DataFrame with shape: (<Delayed>, 8) (3D points)
├── Shapes
│     ├── 'cell_boundaries': GeoDataFrame shape: (167780, 1) (2D shapes)
│     └── 'cell_circles': GeoDataFrame shape: (167780, 2) (2D shapes)
└── Tables
      └── 'table': AnnData (167780, 313)
with coordinate systems:
    ▸ 'global', with elements:
        morphology_focus (Images), morphology_mip (Images), transcripts (Points), cell_boundaries (Shapes), cell_circles (Shapes)

In [24]:
sdata.points['transcripts'].shape[0].compute()

42638083

In [ ]:
xi = df["X"].astype(np.uint32)
yi = df["Y"].astype(np.uint32)
codes = morton_interleave(xi, yi, bits=32)   # 64-bit Morton code

In [113]:
import pandas as pd
import numpy as np
from spatialdata import get_element_annotators

In [41]:
ddf = sdata.points['transcripts']

In [143]:
[x_min, x_max, y_min, y_max] = [ddf["x"].min().compute(), ddf["x"].max().compute(), ddf["y"].min().compute(), ddf["y"].max().compute()]

In [33]:
df = sdata.points['transcripts'].head(10)
df

,x,y,z,feature_name,cell_id,overlaps_nucleus,transcript_id,qv
0,4.395842,328.666473,12.019493,SEC11C,565,0,281474976710656,18.662479
1,5.074415,236.964844,7.608510,NegControlCodeword_0502,540,0,281474976710657,18.634956
2,4.702023,322.797150,12.289083,SEC11C,562,0,281474976710658,18.662479
3,4.906601,581.428650,11.222615,DAPK3,271,0,281474976710659,20.821745
4,5.660699,720.851746,9.265523,TCIM,291,0,281474976710660,18.017488
5,5.899098,748.592773,9.818688,TCIM,297,0,281474976710661,18.017488
6,6.249354,219.854141,10.271250,NKG7,536,0,281474976710662,40.000000
7,7.776000,878.157532,12.464459,RAPGEF3,1089,0,281474976710663,20.488186
8,6.397148,232.495712,7.837698,PPARG,540,0,281474976710664,35.338028
9,6.493312,211.362808,10.820307,RAPGEF3,532,0,281474976710665,40.000000


In [45]:
MORTON_CODE_NUM_BITS = 32 # Resulting morton codes will be stored as uint32.
MORTON_CODE_VALUE_MIN = 0
MORTON_CODE_VALUE_MAX = 2**(MORTON_CODE_NUM_BITS/2) - 1

In [46]:
MORTON_CODE_VALUE_MAX

65535.0

In [80]:
def norm_series_to_uint(series, v_min, v_max):
    """
    Scale numeric Series (int or float) to integer grid [0, 2^bits-1], handling NaNs.
    """
    # Cast to float64
    series_f64 = series.astype("float64")
    # Normalize the array values to be between 0.0 and 1.0
    norm_series_f64 = (series_f64 - v_min) / (v_max - v_min)
    # Clip to ensure no values are outside 0/1 range
    clipped_norm_series_f64 = np.clip(norm_series_f64, 0.0, 1.0)
    # Multiply by the morton code max-value to scale from [0,1] to [0,65535]
    out = (clipped_norm_series_f64 * MORTON_CODE_VALUE_MAX).astype(np.uint32)
    # Set NaNs to 0.
    out = out.fillna(0)
    return out

In [81]:
def norm_ddf_to_uint(ddf):
    [x_min, x_max, y_min, y_max] = [ddf["x"].min().compute(), ddf["x"].max().compute(), ddf["y"].min().compute(), ddf["y"].max().compute()]
    ddf["x_uint"] = norm_series_to_uint(ddf["x"], x_min, x_max)
    ddf["y_uint"] = norm_series_to_uint(ddf["y"], y_min, y_max)
    return ddf

In [146]:
def _part1by1_16(x):
    """
    Spread each 16-bit value into 32 bits by inserting zeros between bits.
    Input:  uint32 array (values must fit in 16 bits)
    Output: uint32 array (bit-spread)
    """
        
    assert x.dtype.name == 'uint32'
    
    # Mask away any bits above 16 (just in case input wasn't clean).
    x = x & np.uint32(0x0000FFFF)
    
    # First spread: shift left by 8 bits, OR with original, then mask.
    # After this, groups of 8 bits are separated by 8 zeros.
    # x = (x | (x << 8)) & np.uint32(0x00FF00FF)
    x = (x | np.left_shift(x, 8)) & np.uint32(0x00FF00FF)
    
    # Spread further: now groups of 4 bits separated by 4 zeros.
    x = (x | np.left_shift(x, 4)) & np.uint32(0x0F0F0F0F)
    
    # Spread further: groups of 2 bits separated by 2 zeros.
    x = (x | np.left_shift(x, 2)) & np.uint32(0x33333333)
    
    # Final spread: single bits separated by a zero bit.
    # Now each original bit is in every other position (positions 0,2,4,...).
    x = (x | np.left_shift(x, 1)) & np.uint32(0x55555555)
    
    return x

"""
def _part1by1_32(u32):
    #Spread each 32-bit value into 64 bits by inserting zeros between bits.
    #Input:  uint64 array (values must fit in 32 bits)
    #Output: uint64 array (bit-spread)

    # Mask away any bits above 32 (safety).
    x = u32.astype(np.uint64) & np.uint64(0x00000000FFFFFFFF)
    
    # First spread: separate into 16-bit chunks spaced out.
    x = (x | (x << 16)) & np.uint64(0x0000FFFF0000FFFF)
    
    # Spread further: each 8-bit chunk separated.
    x = (x | (x << 8)) & np.uint64(0x00FF00FF00FF00FF)
    
    # Spread further: each 4-bit nibble separated.
    x = (x | (x << 4)) & np.uint64(0x0F0F0F0F0F0F0F0F)
    
    # Spread further: 2-bit groups separated.
    x = (x | (x << 2)) & np.uint64(0x3333333333333333)
    
    # Final spread: single bits separated by zeros.
    # Now each original bit occupies every other position (0,2,4,...).
    x = (x | (x << 1)) & np.uint64(0x5555555555555555)
    
    return x
"""

def morton_interleave(ddf):
    """
    Vectorized Morton interleave for integer arrays xi, yi
    already scaled to [0, 2^bits - 1].
    Returns Morton codes as uint32 (if bits<=16) or uint64 (if bits<=32).
    """
    
    xi = ddf["x_uint"]
    yi = ddf["y_uint"]
    
    # Spread x and y bits into even (x) and odd (y) positions.
    xs = _part1by1_16(xi)
    ys = _part1by1_16(yi)

    # Interleave: shift y bits left by 1 so they go into odd positions,
    # then OR with x bits in even positions.
    code = np.left_shift(ys.astype(np.uint64), 1) | xs.astype(np.uint64)
        
    # Fits in 32 bits since we only had 16+16 input bits.
    return code.astype(np.uint32)

def sdata_morton_sort_points(sdata, element):
    ddf = sdata.points[element]
    
    # Compute morton codes
    ddf = norm_ddf_to_uint(ddf)
    ddf["morton_code_2d"] = morton_interleave(ddf)
    
    
    
    if "z" in ddf.columns:
        num_unique_z = ddf["z"].unique().shape[0].compute()
        if num_unique_z < 100:
            # Heuristic for interpreting the 3D data as 2.5D
            # Reference: https://github.com/scverse/spatialdata/issues/961
            sorted_ddf = ddf.sort_values(by=["z", "morton_code_2d"], ascending=True)
        else:
            # TODO: include z as a dimension in the morton code in the 3D case?
            
            # For now, just return the data sorted by 2D code.
            sorted_ddf = ddf.sort_values(by="morton_code_2d", ascending=True)
    else:
        sorted_ddf = ddf.sort_values(by="morton_code_2d", ascending=True)
    sdata.points[element] = sorted_ddf
    
    annotating_tables = get_element_annotators(sdata, element)
    
    # TODO: Sort any annotating table(s) as well.
    
    return sdata

In [137]:
sdata.points["transcripts"]["z"].unique().shape[0].compute()

np.int64(14983003)

In [130]:
sdata_morton_sort_points(sdata, "transcripts")

/Users/mkeller/research/dbmi/vitessce/vitessce-python/.venv/lib/python3.10/site-packages/spatialdata/_core/_elements.py:115: UserWarning: Key `transcripts` already exists. Overwriting it in-memory.
  self._check_key(key, self.keys(), self._shared_keys)


SpatialData object, with associated Zarr store: /Users/mkeller/research/dbmi/vitessce/vitessce-python/docs/notebooks/data/xenium_rep1_io.spatialdata.zarr
├── Images
│     ├── 'morphology_focus': DataTree[cyx] (1, 25778, 35416), (1, 12889, 17708), (1, 6444, 8854), (1, 3222, 4427), (1, 1611, 2213)
│     └── 'morphology_mip': DataTree[cyx] (1, 25778, 35416), (1, 12889, 17708), (1, 6444, 8854), (1, 3222, 4427), (1, 1611, 2213)
├── Points
│     └── 'transcripts': DataFrame with shape: (<Delayed>, 12) (3D points)
├── Shapes
│     ├── 'cell_boundaries': GeoDataFrame shape: (167780, 1) (2D shapes)
│     └── 'cell_circles': GeoDataFrame shape: (167780, 2) (2D shapes)
└── Tables
      └── 'table': AnnData (167780, 313)
with coordinate systems:
    ▸ 'global', with elements:
        morphology_focus (Images), morphology_mip (Images), transcripts (Points), cell_boundaries (Shapes), cell_circles (Shapes)

### Perform rectangle range queries

Given a query rectangle region (x_0, y_0) to (x_1, y_1):

Steps:

0. Knowledge of MORTON_CODE_NUM_BITS (and by extension MORTON_CODE_VALUE_MIN/MORTON_CODE_VALUE_MAX). 
1. `O(2)`: Get the first and last rows of the dataframe, to identify (x_min, y_min) and (x_max, y_max) respectively. These values are needed for normalization.
    - Update: this does not work. The first point is only guaranteed to have y_min, but its X coordinate may be any value. The last row seems to not be guaranteed to have either x_max or y_max.
2. `O(?)`: Get the morton code value intervals covering the query rectangle region (in morton code space).
3. `O(num_intervals * log(N))`: For each morton code (morton_start, morton_end) interval, perform binary search to identify the corresponding table row ranges.
4. Concatenate the table rows in the resulting table row ranges.
5. If the rectangle covering is loose (as opposed to exact), filter the resulting rows to only those in the query rectangle region.

In [132]:
# Convert a coordinate from the normalized [0, 65535] space to the original space.
def norm_coord_to_orig_coord(norm_coord, orig_x_min, orig_x_max, orig_y_min, orig_y_max):
    [norm_x, norm_y] = norm_coord
    orig_x_range = orig_x_max - orig_x_min
    orig_y_range = orig_y_max - orig_y_min
    return [
        (orig_x_min + (norm_x / MORTON_CODE_VALUE_MAX) * orig_x_range),
        (orig_y_min + (norm_y / MORTON_CODE_VALUE_MAX) * orig_y_range),
    ]

# Convert a coordinate from the original space to the [0, 65535] normalized space.
def orig_coord_to_norm_coord(orig_coord, orig_x_min, orig_x_max, orig_y_min, orig_y_max):
    [orig_x, orig_y] = orig_coord
    orig_x_range = orig_x_max - orig_x_min
    orig_y_range = orig_y_max - orig_y_min
    return [
        ((orig_x - orig_x_min) / orig_x_range) * MORTON_CODE_VALUE_MAX,
        ((orig_y - orig_y_min) / orig_y_range) * MORTON_CODE_VALUE_MAX,
    ]

In [141]:
# --------------------------
# Quadtree / Z-interval helpers
# --------------------------

from typing import Tuple, List, Optional

def intersects(ax0:int, ay0:int, ax1:int, ay1:int,
               bx0:int, by0:int, bx1:int, by1:int) -> bool:
    """Axis-aligned box intersection (inclusive integer bounds)."""
    return not (ax1 < bx0 or bx1 < ax0 or ay1 < by0 or by1 < ay0)

def contained(ix0:int, iy0:int, ix1:int, iy1:int,
              ox0:int, oy0:int, ox1:int, oy1:int) -> bool:
    """Is inner box entirely inside outer box? (inclusive integer bounds)"""
    return (ox0 <= ix0 <= ix1 <= ox1) and (oy0 <= iy0 <= iy1 <= oy1)

def point_inside(x:int, y:int, rx0:int, ry0:int, rx1:int, ry1:int) -> bool:
    return (rx0 <= x <= rx1) and (ry0 <= y <= ry1)

def cell_range(prefix: int, level: int, bits: int) -> Tuple[int, int]:
    """
    All Morton codes in a quadtree cell share the same prefix (2*level bits).
    Fill the remaining lower bits with 0s (lo) or 1s (hi).
    """
    shift = 2 * (bits - level)
    lo = prefix << shift
    hi = ((prefix + 1) << shift) - 1
    return lo, hi

def merge_adjacent(intervals: List[Tuple[int,int]]) -> List[Tuple[int,int]]:
    """Merge overlapping or directly adjacent intervals."""
    if not intervals:
        return []
    intervals.sort(key=lambda t: t[0])
    merged = [intervals[0]]
    for lo, hi in intervals[1:]:
        mlo, mhi = merged[-1]
        if lo <= mhi + 1:
            merged[-1] = (mlo, max(mhi, hi))
        else:
            merged.append((lo, hi))
    return merged

# --------------------------
# Rectangle -> list of Morton intervals
# --------------------------

def zcover_rectangle(rx0:int, ry0:int, rx1:int, ry1:int, bits:int,
                     stop_level: Optional[int] = None,
                     merge: bool = True) -> List[Tuple[int,int]]:
    """
    Compute a (near-)minimal set of Morton code ranges covering the rectangle
    [rx0..rx1] x [ry0..ry1] on an integer grid [0..2^bits-1]^2.

    - If stop_level is None: exact cover (descend to exact containment).
    - If stop_level is set (0..bits): stop descending at that level, adding
      partially-overlapping cells as whole ranges (superset cover).
    """
    if not (0 <= rx0 <= rx1 <= (1<<bits)-1 and 0 <= ry0 <= ry1 <= (1<<bits)-1):
        raise ValueError("Rectangle out of bounds for given bits.")

    intervals: List[Tuple[int,int]] = []

    # stack entries: (prefix, level, xmin, ymin, xmax, ymax)
    stack = [(0, 0, 0, 0, (1<<bits)-1, (1<<bits)-1)]

    while stack:
        prefix, level, xmin, ymin, xmax, ymax = stack.pop()

        if not intersects(xmin, ymin, xmax, ymax, rx0, ry0, rx1, ry1):
            continue

        # If we stop at this level for a loose cover, add full cell range.
        if stop_level is not None and level == stop_level:
            intervals.append(cell_range(prefix, level, bits))
            continue

        # Fully contained: add full cell range.
        if contained(xmin, ymin, xmax, ymax, rx0, ry0, rx1, ry1):
            intervals.append(cell_range(prefix, level, bits))
            continue

        # Leaf cell: single lattice point (only happens when level==bits)
        if level == bits:
            if point_inside(xmin, ymin, rx0, ry0, rx1, ry1):
                intervals.append(cell_range(prefix, level, bits))
            continue

        # Otherwise, split into 4 children (Morton order: 00,01,10,11)
        midx = (xmin + xmax) // 2
        midy = (ymin + ymax) // 2

        # q0: (x<=midx, y<=midy) -> child code 0b00
        stack.append(((prefix << 2) | 0,
                      level+1,
                      xmin, ymin, midx, midy))
        # q1: (x>midx, y<=midy)  -> child code 0b01
        stack.append(((prefix << 2) | 1,
                      level+1,
                      midx+1, ymin, xmax, midy))
        # q2: (x<=midx, y>midy)  -> child code 0b10
        stack.append(((prefix << 2) | 2,
                      level+1,
                      xmin, midy+1, midx, ymax))
        # q3: (x>midx, y>midy)   -> child code 0b11
        stack.append(((prefix << 2) | 3,
                      level+1,
                      midx+1, midy+1, xmax, ymax))

    return merge_adjacent(intervals) if merge else intervals

In [153]:
import dask.dataframe as dd

# Construct dask dataframe of points in range 100x200:

toy_df = pd.DataFrame(index=[], data=[], columns=["x", "y"])
toy_df["x"] = np.random.uniform(low=0.0, high=100.0, size=20)
toy_df["y"] = np.random.uniform(low=0.0, high=200.0, size=20)

toy_ddf = dd.from_pandas(toy_df, npartitions=2)

In [166]:
# Compute morton codes
toy_ddf = norm_ddf_to_uint(toy_ddf)
toy_ddf["morton_code_2d"] = morton_interleave(toy_ddf)
sorted_ddf = toy_ddf.sort_values(by="morton_code_2d", ascending=True).compute()

In [167]:
sorted_ddf.reset_index()

,index,x,y,x_uint,y_uint,morton_code_2d
0,10,18.066939,1.769065,10808,0,71566656
1,18,2.847571,26.880494,588,8741,135010418
2,5,10.322758,44.756470,5608,14964,194608736
3,9,46.157966,44.731097,29671,14955,529366175
4,0,22.064498,79.399301,13492,27023,764593594
5,1,33.631887,59.348116,21260,20043,833429722
6,7,55.847609,36.496019,36178,12088,1224416132
7,15,63.455714,43.365232,41286,14479,1317114046
8,8,94.708812,14.841509,62273,4550,1460121641
9,17,97.890740,25.150878,64409,8139,1475338699


In [158]:
orig_rect = [[0, 0], [100, 100]] # x0, y0, x1, y1
norm_rect = [
    orig_coord_to_norm_coord(orig_rect[0], orig_x_min=0, orig_x_max=100, orig_y_min=0, orig_y_max=200),
    orig_coord_to_norm_coord(orig_rect[1], orig_x_min=0, orig_x_max=100, orig_y_min=0, orig_y_max=200)
]

morton_intervals = zcover_rectangle(rx0 = norm_rect[0][0], ry0 = norm_rect[0][1], rx1 = norm_rect[1][0], ry1 = norm_rect[1][1], bits = 16, stop_level = None, merge = True)
morton_intervals

[(0, 2147483647)]

In [159]:
orig_rect = [[0, 0], [50, 50]] # x0, y0, x1, y1
norm_rect = [
    orig_coord_to_norm_coord(orig_rect[0], orig_x_min=0, orig_x_max=100, orig_y_min=0, orig_y_max=200),
    orig_coord_to_norm_coord(orig_rect[1], orig_x_min=0, orig_x_max=100, orig_y_min=0, orig_y_max=200)
]


morton_intervals = zcover_rectangle(rx0 = norm_rect[0][0], ry0 = norm_rect[0][1], rx1 = norm_rect[1][0], ry1 = norm_rect[1][1], bits = 16, stop_level = None, merge = True)
morton_intervals

[(0, 536870911)]

In [161]:
orig_rect = [[50, 50], [100, 150]] # x0, y0, x1, y1
norm_rect = [
    orig_coord_to_norm_coord(orig_rect[0], orig_x_min=0, orig_x_max=100, orig_y_min=0, orig_y_max=200),
    orig_coord_to_norm_coord(orig_rect[1], orig_x_min=0, orig_x_max=100, orig_y_min=0, orig_y_max=200)
]


morton_intervals = zcover_rectangle(rx0 = norm_rect[0][0], ry0 = norm_rect[0][1], rx1 = norm_rect[1][0], ry1 = norm_rect[1][1], bits = 16, stop_level = None, merge = True)
morton_intervals

[(1610612736, 2147483647), (3221225472, 3758096383)]

In [164]:
from bisect import bisect_left, bisect_right
# --------------------------
# Morton intervals -> row ranges in a Morton-sorted column
# --------------------------

def zquery_rows(morton_sorted: List[int], intervals: List[Tuple[int,int]]) -> List[Tuple[int,int]]:
    """
    For each Z-interval [zlo, zhi], binary-search in the sorted Morton column
    and return row index half-open ranges [i, j) to scan.
    """
    ranges: List[Tuple[int,int]] = []
    for zlo, zhi in intervals:
        i = bisect_left(morton_sorted, zlo)
        j = bisect_right(morton_sorted, zhi)
        if i < j:
            ranges.append((i, j))
    return ranges

In [165]:
morton_sorted = sorted_ddf["morton_code_2d"].compute().values.tolist()
zquery_rows(morton_sorted, morton_intervals)

[(11, 13), (17, 19)]

In [134]:
sdata.points["transcripts"].head()

,x,y,z,feature_name,cell_id,overlaps_nucleus,transcript_id,qv,x_uint,y_uint,morton_code,morton_code_2d
822656,51.057068,963.561218,2.436733,BLANK_0364,23902,0,281530811809758,10.209187,460,11493,144832626,144832626
1100692,130.295517,1822.821655,3.261316,SERPINA3,27633,1,281530812101000,14.430645,1151,21789,573708279,573708279
798858,15.582742,1763.884155,3.292289,TPD52,1648,1,281530811784852,10.131371,152,21083,570975178,570975178
685576,52.254360,1792.832642,3.354824,GATA3,28965,0,281530811666202,18.372648,471,21430,571202365,571202365
2491977,452.574585,1937.871948,3.553060,ANKRD30A,27205,1,281586646320655,19.528357,3957,23168,584946961,584946961


In [122]:
get_element_annotators(sdata, "transcripts")

set()

In [79]:
ddf["y_uint"].compute()

0          3885
1          2786
2          3815
3          6914
4          8584
          ...  
638078    63402
638079    57379
638080    64958
638081    57940
638082    56730
Name: y_uint, Length: 42638083, dtype: uint64

In [25]:
sdata.points['transcripts']

,x,y,z,feature_name,cell_id,overlaps_nucleus,transcript_id,qv
npartitions=8,,,,,,,,
,float32,float32,float32,category[unknown],int32,uint8,uint64,float32
,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...


## Configure Vitessce

Vitessce needs to know which pieces of data we are interested in visualizing, the visualization types we would like to use, and how we want to coordinate (or link) the views.

In [ ]:
vc = VitessceConfig(
    schema_version="1.0.18",
    name='MERFISH SpatialData Demo',
)
# Add data to the configuration:
wrapper = SpatialDataWrapper(
    sdata_path=spatialdata_filepath,
    # The following paths are relative to the root of the SpatialData zarr store on-disk.
    image_path="images/rasterized",
    table_path="tables/table",
    obs_feature_matrix_path="tables/table/X",
    obs_spots_path="shapes/cells",
    coordinate_system="global",
    coordination_values={
        # The following tells Vitessce to consider each observation as a "spot"
        "obsType": "cell",
    }
)
dataset = vc.add_dataset(name='MERFISH').add_object(wrapper)

# Add views (visualizations) to the configuration:
spatial = vc.add_view("spatialBeta", dataset=dataset)
feature_list = vc.add_view("featureList", dataset=dataset)
layer_controller = vc.add_view("layerControllerBeta", dataset=dataset)
obs_sets = vc.add_view("obsSets", dataset=dataset)

vc.link_views_by_dict([spatial, layer_controller], {
    'spotLayer': CL([{
        'obsType': 'cell',
    }]),
}, scope_prefix=get_initial_coordination_scope_prefix("A", "obsSpots"))

vc.link_views([spatial, layer_controller, feature_list, obs_sets], ['obsType'], [wrapper.obs_type_label])

# Layout the views
vc.layout(spatial | (feature_list / layer_controller / obs_sets));

### Render the widget

In [ ]:
vw = vc.widget()
vw